In [11]:
#%pip install xgboost

     --------------------------------------- 89.1/89.1 MB 12.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [152]:
import mediapipe as mp
import cv2
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from  xgboost import XGBClassifier
import xgboost
from sklearn.metrics import r2_score
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
def calculate_angles(a,b,c):
    a = np.array(a) #first
    b = np.array(b) 
    c = np.array(c) #last

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360- angle

    return angle

In [ ]:
#Saves everything that actually comes out of the pictures
#x,y,z,visibility,labal are 1 picture
#The library is responsible for taking a picture and pasting the link
#
data = {
    'x': [],    #Knee angle
    'y': [],    #Elbow angle
    'z': [],    #LEFT AND RIGHT,depth
    'visibility': [],#How accurate the spot detection is and if not exsist(0)
    'label': [] #The value of the character in terms of the X-axis
}

test_data = {
    'x': [],
    'y': [],
    'z': [],
    'visibility': [],
    'label': []
}

In [154]:
#Runs over the entire folder of images 
# assign directory
directory = r'C:\Users\mosac\Git Repositories\FitnessAI\images'
label = ''
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
    
        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            #run over the images inside the folders
            #takes every image and put it inside a frame,f=image
            frame = cv2.imread(f, cv2.IMREAD_ANYCOLOR)#read the image to frame of cv2

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)#paint the pose
            image.flags.writeable = False
      
            results = pose.process(image)#the picther with the scale and dots

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#change the color format

            #Extract  the dots from the image,every dots contain x,y,z,visabillity and saved in arr
            try:
                landmarks = results.pose_landmarks.landmark
            except:
                pass


            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                            )

        if 'deadlift' in f:
            label = 'deadlift'
        elif 'squat' in f:
            label = 'squat'
        elif 'bench' in f:
            label = 'bench'
        else:
            continue
        
        
        #calculate desired angels.
        #add those angels values to the data array.

        #Can Be Deleted
        for i in range(11,len(mp_pose.PoseLandmark)-10):
            data["x"].append(landmarks[i].x)
            data["y"].append(landmarks[i].y)
            data["z"].append(landmarks[i].z)
            data["visibility"].append(landmarks[i].visibility)
            data["label"].append(label)


Squat Test Data

In [155]:

# assign directory
directory = r'C:\Users\mosac\Git Repositories\FitnessAI\Test_Squats'
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
            frame = cv2.imread(f, cv2.IMREAD_ANYCOLOR)

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
      
            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


            try:
                landmarks = results.pose_landmarks.landmark
            except:
                pass


            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                            )

        label = 'squat'
      
        
        for i in range(11,len(mp_pose.PoseLandmark)-10):
            test_data["x"].append(landmarks[i].x)
            test_data["y"].append(landmarks[i].y)
            test_data["z"].append(landmarks[i].z)
            test_data["visibility"].append(landmarks[i].visibility)
            test_data["label"].append(label)

Deadlift Test Data

In [116]:
test_data = {
    'x': [],
    'y': [],
    'z': [],
    'visibility': [],
    'label': []
}

# assign directory
directory = r'C:\Users\mosac\Git Repositories\FitnessAI\Test_deadlift'
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
            frame = cv2.imread(f, cv2.IMREAD_ANYCOLOR)

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
      
            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


            try:
                landmarks = results.pose_landmarks.landmark
            except:
                pass


            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                            )

        label = 'deadlift'
      
        
        for i in range(11,len(mp_pose.PoseLandmark)-10):
            test_data["x"].append(landmarks[i].x)
            test_data["y"].append(landmarks[i].y)
            test_data["z"].append(landmarks[i].z)
            test_data["visibility"].append(landmarks[i].visibility)
            test_data["label"].append(label)

In [89]:
data['visibility'][0]

0.9998136162757874

In [156]:
#Make data array into pandas DataFrame
df = pd.DataFrame(data=data,columns=data.keys())
df_test = pd.DataFrame(data=test_data,columns=test_data.keys())

In [157]:
#look for null values
total_nulls = df.isnull().sum().sum()
print(total_nulls)

total_nulls = df_test.isnull().sum().sum()
print(total_nulls)
#0 nulls found

0
0


In [158]:
print(df.groupby(['label'])['label'].count())
print(df_test.groupby(['label'])['label'].count())
#print(df[df.loc[:,'label']=='squat'])
#print(df_test.tail())

label
bench       3528
deadlift    2592
squat       4536
Name: label, dtype: int64
label
squat    36
Name: label, dtype: int64


In [160]:
#Encoding categorial features
le = LabelEncoder()
le.fit(df.loc[:,'label'])
df.loc[:,'label'] = le.transform(df.loc[:,'label'])
df.tail()

df_test.loc[:,'label'] = le.transform(df_test.loc[:,'label'])
df_test.tail()

,x,y,z,visibility,label
31,0.468979,0.403448,0.392113,0.056002,2
32,0.440374,0.370204,0.117034,0.407221,2
33,0.475659,0.399690,0.374155,0.060266,2
34,0.445132,0.373112,0.131432,0.430681,2
35,0.479795,0.402587,0.373305,0.063188,2


In [161]:
print(df.head())
print(df.tail())
print(df_test.tail())

          x         y         z  visibility  label
0  0.595476  0.340851 -0.318189    0.999814      0
1  0.540570  0.333555  0.161531    0.999434      0
2  0.532153  0.205141 -0.490691    0.991011      0
3  0.488872  0.238865  0.287628    0.511165      0
4  0.486908  0.080763 -0.505209    0.968490      0
              x         y         z  visibility  label
10651  0.346274  0.475674 -0.078109    0.048087      2
10652  0.352617  0.411087 -0.032660    0.083758      2
10653  0.345670  0.467875 -0.079193    0.049886      2
10654  0.354187  0.413280 -0.024628    0.074859      2
10655  0.346774  0.465587 -0.077620    0.047159      2
           x         y         z  visibility  label
31  0.468979  0.403448  0.392113    0.056002      2
32  0.440374  0.370204  0.117034    0.407221      2
33  0.475659  0.399690  0.374155    0.060266      2
34  0.445132  0.373112  0.131432    0.430681      2
35  0.479795  0.402587  0.373305    0.063188      2


In [ ]:
data

In [162]:
indeces=[]
for i in range(len(data['visibility'])):
    if data['visibility'][i] < 0.9:
        indeces.append(i)

In [163]:
print(len(df))
df = df.drop(index=indeces)
print(len(df))

10656
6408


In [ ]:
#Outlier handling
"""
for i in range(len(df.columns)-5):
  col_name = df.columns[i]
  col = df[col_name]
  Q1 = col.quantile(0.25)
  Q3 = col.quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 2 * IQR
  upper_bound = Q3 + 2 * IQR

  # Replace values outside the bounds with the bound values
  col_winsorized = col.clip(lower_bound, upper_bound)

  # Replace the original column with the winsorized data
  df[col_name] = col_winsorized
"""

In [164]:
from sklearn.utils import shuffle

X_train = df.drop(['label','visibility'],axis=1)
y_train = df['label']

X_test = df_test.drop(['label','visibility'],axis=1)
y_test = df_test['label']

X_test,y_test = shuffle(X_test,y_test)
X_train,y_train = shuffle(X_train,y_train)


In [165]:
print(X_train)
print(len(y_test))

              x         y         z
10121  0.381948  0.437174 -0.741139
1677   0.289193  0.350661  0.215509
446    0.539378  0.488278 -0.239149
1467   0.358232  0.386019 -0.676417
2      0.532153  0.205141 -0.490691
...         ...       ...       ...
7510   0.604157  0.284615 -0.763245
1393   0.418881  0.619507  0.390456
9633   0.487716  0.186091 -0.991793
10515  0.532691  0.416727 -0.076315
8008   0.641916  0.199597 -0.746113

[6408 rows x 3 columns]
36


In [171]:
xgb_model = XGBClassifier(learning_rate=1, n_estimators= 100,max_depth = 5)

fit = xgb_model.fit(X_train,y_train)

fit_pred = fit.predict(X_train)

xgb_predictions = xgb_model.predict(X_test)

xgb_accuracy = accuracy_score(y_test,xgb_predictions)

#Evaluation - calculating accuracy
xgb_train_acc = accuracy_score(y_train,fit_pred)

#Print accuracy of train and test
print(f"Train Accuracy Is: {round(xgb_train_acc*100,2)}\nTest Accuracy Is: {round(xgb_accuracy*100,2)}")

print(xgb_predictions)

Train Accuracy Is: 99.41
Test Accuracy Is: 30.56
[0 0 0 1 0 1 0 0 0 1 1 2 2 0 0 1 2 2 2 1 1 1 2 0 0 0 2 2 0 0 2 0 2 0 2 0]
